# Rule: **add_electricity**

**Outputs**

- resources/networks/`elec.nc`

In [ ]:
######################################## Parameters

### Run
name = '00_ref'
prefix = ''

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


##### Region files
file_regions_onshore = f'regions_onshore.geojson'
file_regions_offshore = f'regions_offshore.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'
gdf_regions_onshore = gpd.read_file(path_regions+file_regions_onshore)
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)


##### NUTS files (must contain at least columns 'NUTS_ID' and 'geometry')
file_NUTS2 = 'NUTS2_ES.geojson'
file_NUTS3 = 'NUTS3_ES.geojson'
path_NUTS = f'{params['rootpath']}/data_ES/nuts/'
gdf_NUTS2 = gpd.read_file(path_NUTS+file_NUTS2)
gdf_NUTS3 = gpd.read_file(path_NUTS+file_NUTS3)


## `elec.nc`

Load the network and show its components.

In [ ]:
file = f'elec.nc'
path = f'{params['rootpath']}/resources/{prefix}/{name}/networks/'

n = pypsa.Network(path+file)

n

Plot the network.

In [ ]:
#################### Parameters
line_widths = 1*n.lines.s_nom / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, line_widths=line_widths, bus_sizes=params['bus_sizes'], bus_colors=params['bus_colors'], boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Variable: `n.carriers`

Carriers other than AC and DC have been included.

Place `n.carriers` in a dataFrame and show its content.

In [ ]:
cc = n.carriers

cc.head()

### Variable: `n.generators`

Place `n.generators` in a dataFrame and show its content.

In [ ]:
gg = n.generators

gg.head()

#### Summary

What is the aggregated capacity per carrier? 

How many buses have generators for each carrier? How many of them have zero capacity?

In [ ]:
gg.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Buses=pd.NamedAgg(column='p_nom', aggfunc='size'),
    Buses_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x == 0).sum()),
    Buses_non_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x != 0).sum()),
)

#### Maps

Plot a map showing a particular feature of a generation carrier at each region.

Then, show another map with the same info aggregated at some NUTS level.

In [ ]:
#################### Parameters

### Select carrier
carrier = 'onwind'

### Select feature (uncomment one of the following):
# feature = 'area' 
feature = 'p_nom'
# feature = 'p_nom_density'
# feature = 'p_nom_max'
# feature = 'p_nom_max_density'
#feature = 'p_nom_max_ratio'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define gdf_regions
if 'off' in carrier:
    gdf_regions = gdf_regions_offshore
    is_offshore = True
else:
    gdf_regions = gdf_regions_onshore
    is_offshore = False


### Add regions
xp.map_add_region(ax, gdf_regions, params['map_add_region'], is_offshore=is_offshore)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_generators(carrier, n, feature, ax, gdf_regions, params['map_network_generators'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 3



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add regions
xp.map_add_region(ax, gdf_regions, params['map_add_region'], is_offshore=is_offshore)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_generators(carrier, n, f'{feature}_NUTS', ax, gdf_regions, gdf_NUTS, params['map_NUTS_generators'], params_local)

### Variable: `n.lines`

Changes:

- New lines defined in TYNDP may have been incorporated.
- Columns 'project_status' and 'tags' have been included
- Column 'capital_cost' has been filled


Place `n.lines` in a dataFrame and show its content.

In [ ]:
ln = n.lines

ln.head()

How is the relationship between line capital costs and line length?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ln.plot(ax=ax, kind='scatter', x='length', y='capital_cost')

ax.set_xlabel('km')
ax.set_ylabel('EUR')
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.links`

Changes:

- Column 'capital_cost' has been filled


Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()

How is the relationship between link capital costs and link length?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


lk.plot(ax=ax, kind='scatter', x='length', y='capital_cost')

ax.set_xlabel('km')
ax.set_ylabel('EUR')
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.loads_t[p_set]`

Place `n.loads_t[p_set]` in a dataFrame and show its content.

In [ ]:
lot_pset = n.loads_t['p_set']

lot_pset.head()

#### Time series

How do the load time series look like?

In [ ]:
#################### Parameters
start = '2022-03-01'
end = '2022-03-10'



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)

lot_pset.loc[start:end].plot(ax=ax, alpha=.4, legend=False, linewidth=.5)

ax.grid(True, linestyle='--', alpha=1)
ax.set_ylabel('MW')

#### Maps

Plot a map showing a particular feature of the load at each region.

In [ ]:
#################### Parameters

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'annual_load'
feature = 'annual_load_density'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add regions
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_loads_t(n, feature, ax, gdf_regions_onshore, params['map_network_loads_t'], params_local)

### Variable: `n.storage_units`

Place `n.generators` in a dataFrame and show its content.

In [ ]:
su = n.storage_units

su.head()

#### Summary

What is the aggregated capacity per carrier? 

How many buses have storage units for each carrier? How many of them have zero capacity?

In [ ]:
su.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Buses=pd.NamedAgg(column='p_nom', aggfunc='size'),
    Buses_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x == 0).sum()),
    Buses_non_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x != 0).sum()),
)

#### Maps

Plot a map showing a particular feature of a storage unit carrier at each region.

In [ ]:
#################### Parameters

### Select carrier
carrier = 'hydro'

### Select feature (uncomment one of the following):
# feature = 'area' 
feature = 'p_nom'
# feature = 'p_nom_density'
# feature = 'max_hours'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add regions
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_storage_units(carrier, n, feature, ax, gdf_regions_onshore, params['map_network_storage_units'], params_local)

#### Maximum hours

What is the relation between installed capacity and max. hours, for each carrier?

In [ ]:
#################### Figure
fig_size = [8,4]
fig, ax = plt.subplots(figsize=fig_size)

tech_colors = n.carriers['color']

for carrier, group in su.groupby('carrier'):
    ax.scatter(group['p_nom'], group['max_hours'], label=carrier, color=tech_colors[carrier], alpha=0.7)

ax.set_xlabel('Installed capacity [MW]')
ax.set_ylabel('Max. hours')
ax.set_title('Storage units')
ax.grid(True, linestyle='--', alpha=0.5)
ax.legend()